In [ ]:
import numpy as np
import random
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn.utils.rnn import *
import torch.nn.functional as F
from torch.autograd import Variable
# from torchnlp.nn import LockedDropout
# from torchnlp.nn import WeightDrop
# import torch.nn.utils.clip_grad_norm_ as clip_grad_norm_
!pip install python-Levenshtein
from Levenshtein import distance
import pdb

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
!nvidia-smi

     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144800 sha256=4175a92e2fec6b8c62f4fa8d6f33282ab1a3ef17e13e07f33f23ecd9b756dbb6
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
cuda
Sat May  9 00:02:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
speech_train = np.load('/content/drive/My Drive/HW4P2/train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load('/content/drive/My Drive/HW4P2/dev_new.npy', allow_pickle=True, encoding='bytes')
speech_test = np.load('/content/drive/My Drive/HW4P2/test_new.npy', allow_pickle=True, encoding='bytes')
transcript_train = np.load('/content/drive/My Drive/HW4P2/train_transcripts.npy', allow_pickle=True)
transcript_valid = np.load('/content/drive/My Drive/HW4P2/dev_transcripts.npy', allow_pickle=True)

In [ ]:
# all characters
LETTER_LIST = ['<pad>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', \
               'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-', "'", '.', '_', '+', ' ','<sos>','<eos>']

# dicts for later use
letter2index = dict(zip(LETTER_LIST, np.arange(len(LETTER_LIST))))
index2letter = dict(zip(np.arange(len(LETTER_LIST)),LETTER_LIST))

def transform_letter_to_index(transcript, letter2index):
  '''
  Transforms alphabetical input to numerical input, replace each letter by its corresponding
  index from letter_list
  :param transcript :(N, ) Transcripts are the text input
  :param letter_list: Letter list defined above
  :return letter_to_index_list: Returns a list for all the transcript sentence to index
  '''
  character_list = list(' '.join(transcript.astype('str')))
  return [letter2index['<sos>']]+[letter2index[x] for x in character_list]+[letter2index['<eos>']]

def mean_normalization(utterance):
  mu = utterance.mean(axis=0)
  return (utterance - mu + 1e-8)

def my_collate_fn(batch):
  (data, labels) = zip(*batch)
  in_lens = torch.LongTensor([len(i) for i in data])
  target_lens = torch.LongTensor([len(i) for i in labels])
  return pad_sequence(data), pad_sequence(labels, batch_first=True, padding_value=0), in_lens, target_lens

def out_to_strings(out, remove_ends=True):
    indices = torch.argmax(out, axis=2).tolist()
    for i, a in enumerate(indices):
        if letter2index['<eos>'] in a:
            indices[i] = a[:a.index(letter2index['<eos>']) + 1]
    return_this = [''.join(index2letter[x] for x in seq).replace("<sos>", "").replace("<eos>", "").replace("<pad>", "").strip() for seq in indices]
    return return_this

In [ ]:
class Speech2TextDataset(Dataset):
  '''
  Dataset class for the speech to text data
  '''
  def __init__(self, l2i, speech, text=None, isTrain=True, normalize=True):
    if normalize:
      self.speech = [torch.FloatTensor(mean_normalization(df)) for df in speech]
    else:
      self.speech = speech
    self.isTrain = isTrain
    if (text is not None):
        self.text = [torch.LongTensor(transform_letter_to_index(i,l2i)) for i in text]

  def __len__(self):
    return len(self.speech)

  def __getitem__(self, index):
    if (self.isTrain == True):
        return self.speech[index], self.text[index]
    else:
        return torch.tensor(self.speech[index])
train_dataset = Speech2TextDataset(letter2index, speech_train, transcript_train)
valid_dataset = Speech2TextDataset(letter2index, speech_valid, transcript_valid)

In [ ]:

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, collate_fn=my_collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=True, num_workers=8, collate_fn=my_collate_fn)
print("train batches:\t", len(train_loader))
print("dev batches:\t", len(valid_loader))

train batches:	 194
dev batches:	 9


In [ ]:
def locked_dropout(x, training, dropout):
    if not training or not dropout:
        return x
    # same mask repeated across the sequence dimension
    m = x.data.new(1, x.size(1), x.size(2)).bernoulli_(1 - dropout)
    mask = Variable(m, requires_grad=False) / (1 - dropout)
    mask = mask.expand_as(x)
    return mask * x

In [ ]:
# pBLSTM
class pBLSTM(nn.Module):
  '''
  Pyramidal BiLSTM
  The length of utterance (speech input) can be hundereds to thousands of frames long.
  The Paper reports that a direct LSTM implementation as Encoder resulted in slow convergence,
  and inferior results even after extensive training.
  The major reason is inability of AttendAndSpell operation to extract relevant information
  from a large number of input steps.
  '''
  def __init__(self, input_dim, hidden_dim):
    super(pBLSTM, self).__init__()
    # print('input_dim: ', input_dim)
    # print('hidden_dim: ', hidden_dim)
    self.blstm = nn.LSTM(input_size=input_dim, hidden_size=int(hidden_dim), num_layers=1, bidirectional=True)

  def forward(self, x, lens, prob, isTrain):
    '''
    :param x :(N, T) input to the pBLSTM
    :return output: (N, T, H) encoded sequence from pyramidal Bi-LSTM
    '''
    # print('pblstm_input_x: ', x)
    x = self.blstm(x)[0]
    x, _ = pad_packed_sequence(x)

    x = x.permute(1, 0, 2) # -> batch_size, seq, N
    (batch_size, seq, N) = x.size()

    # logic to handle odd seq

    if seq % 2 != 0:
      x = x[:, :-1, :]
    # reduce seq to seq/2
    x = x.contiguous().view(batch_size, seq // 2, 2, N).mean(dim=2)
    # x = x.reshape(batch_size, seq // 2, N * 2)
    lens = lens // 2
    x = x.permute(1, 0, 2) # -> seq, batch_size, N
    if isTrain == True:
        x = locked_dropout(x, isTrain, dropout=prob)
    # packing for next layer
    x = pack_padded_sequence(x, lens, enforce_sorted=False)

    return x, lens

In [ ]:
# Encoder
class Encoder(nn.Module):
  '''
  Encoder takes the utterances as inputs and returns the key and value.
  Key and value are nothing but simple projections of the output from pBLSTM network.
  '''
  def __init__(self, input_dim, hidden_dim, value_size=128,key_size=128, isTrain=True):
    super(Encoder, self).__init__()
    self.conv1 = nn.Conv1d(40, 256 ,kernel_size=2, padding=1, stride=1)
    self.bn1 = nn.BatchNorm1d(256)
    self.bn2 = nn.BatchNorm1d(512)
    self.dropout = nn.Dropout(0.3)
    self.conv2 = nn.Conv1d(256, 512 ,kernel_size=4, padding=1, stride=1)
    self.lstm = nn.LSTM(input_size=512, hidden_size=hidden_dim, num_layers=1, bidirectional=True)

    self.pblstm1 = pBLSTM(hidden_dim * 2, hidden_dim)
    self.pblstm2 = pBLSTM(hidden_dim * 2, hidden_dim)
    self.pblstm3 = pBLSTM(hidden_dim * 2, hidden_dim)

    # self.pblstm1 = pBLSTM(hidden_dim * 2, hidden_dim / 2)
    # self.pblstm2 = pBLSTM(hidden_dim * 2, hidden_dim / 2)
    # self.pblstm3 = pBLSTM(hidden_dim * 2, hidden_dim / 2)
    # self.ld1 = LockedDropout(0.2)
    # self.ld2 = LockedDropout(0.3)

    self.key_network = nn.Linear(hidden_dim*2, value_size)
    self.value_network = nn.Linear(hidden_dim*2, key_size)

  def forward(self, x, lens, isTrain = True):
    # print("Entered Encoder", x.shape)
    x = x.permute(1,2,0).contiguous()
    x = F.relu(self.dropout(self.bn1(self.conv1(x))))
    x = F.relu(self.dropout(self.bn2(self.conv2(x))))
    x = x.permute(2,0,1)
    rnn_inp = pack_padded_sequence(x, lengths=lens, batch_first=False, enforce_sorted=False)
    prnn_inp, _ = self.lstm(rnn_inp)
    prnn_inp1, output_lens = self.pblstm1(prnn_inp, lens, 0.4, isTrain)
    prnn_inp2, output_lens = self.pblstm2(prnn_inp1, output_lens, 0.2, isTrain)
    outputs, output_lens = self.pblstm3(prnn_inp2, output_lens, 0.1, isTrain)
    # print("Second LSTM")
    linear_input, _ = pad_packed_sequence(outputs, batch_first=True) # -> (batch_size, seq , N)

    # linear_input.transpose(1, 0) # -> (batch_size, seq, N)

    keys = self.key_network(linear_input) # (batch_size, seq, key_size)
    value = self.value_network(linear_input) # (batch_size, seq, value_size)

    # print('keys: ', keys.shape)
    # print('value: ', value.shape)
    return keys.transpose(1, 0), value.transpose(1, 0), output_lens


In [ ]:
class Attention(nn.Module):
  '''
  Attention is calculated using key, value and query from Encoder and decoder.
  Below are the set of operations you need to perform for computing attention:
      energy = bmm(key, query)
      attention = softmax(energy)
      mask_attention
      context = bmm(mask_attention, value)
  '''
  def __init__(self):
    super(Attention, self).__init__()

  def forward(self, query, key, value, lens):
    '''
    :param query :(N, context_size) Query is the output of LSTMCell from Decoder
    :param key: (N, key_size) Key Projection from Encoder per time step
    :param value: (N, value_size) Value Projection from Encoder per time step
    :return output: Attended Context
    :return attention_mask: Attention mask that can be plotted
    '''
    key = key.transpose(1, 0) # -> batch_size, seq, key_size
    value = value.transpose(1, 0) # -> batch_size, seq, value_size

    energy = torch.bmm(key, query.unsqueeze(2)).squeeze(2)

    lens = lens.cpu()
    mask = torch.arange(key.shape[1]).unsqueeze(0) >= lens.unsqueeze(1)
    mask.to(device)
    energy[mask] = float('-Inf')

    attention = F.softmax(energy, dim=1)

    context = torch.bmm(attention.unsqueeze(1), value)

    return context.squeeze(1), attention

In [ ]:
def embedded_dropout(embed, words, dropout=0.2, scale=None):
  if dropout:
    mask = embed.weight.data.new().resize_((embed.weight.size(0), 1)).bernoulli_(1 - dropout).expand_as(embed.weight) / (1 - dropout)
    masked_embed_weight = mask * embed.weight
  else:
    masked_embed_weight = embed.weight
  if scale:
    masked_embed_weight = scale.expand_as(masked_embed_weight) * masked_embed_weight

  padding_idx = embed.padding_idx
  if padding_idx is None:
      padding_idx = -1

  X = torch.nn.functional.embedding(words, masked_embed_weight,
    padding_idx, embed.max_norm, embed.norm_type,
    embed.scale_grad_by_freq, embed.sparse
  )
  return X

In [ ]:
class Decoder(nn.Module):
    '''
    each forward call of decoder deals with just one time step,
    thus we use LSTMCell instead of LSLTM here.
    The output from the second LSTMCell can be used as query here for attention module.
    In place of value that we get from the attention, this can be replace by context we get from the attention.
    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.
    '''
    def __init__(self, vocab_size, hidden_dim, value_size=128, key_size=128, isAttended=True):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
        self.lstm1 = nn.LSTMCell(input_size=hidden_dim + value_size, hidden_size=hidden_dim)
        self.lstm2 = nn.LSTMCell(input_size=hidden_dim, hidden_size=key_size)

        self.isAttended = isAttended
        if (isAttended == True):
            self.attention = Attention()

        self.character_prob = nn.Linear(key_size + value_size, vocab_size)

    def forward(self, key, values, lens, text=None, isTrain=True, teach_prob=0.7):
        '''
        :param key :(T, N, key_size) Output of the Encoder Key projection layer
        :param values: (T, N, value_size) Output of the Encoder Value projection layer
        :param text: (N, text_len) Batch input of text with text_length
        :param isTrain: Train or eval mode
        :return predictions: Returns the character perdiction probability
        '''
        batch_size = key.shape[1]

        if (isTrain == True):
          max_len =  text.shape[1]
        #   embeddings = self.embedding(text)
          embeddings = embedded_dropout(self.embedding, text)

        else:
            max_len = 250

        predictions = []
        hidden_states = [None, None]
        prediction = torch.zeros((batch_size,len(LETTER_LIST)), dtype=torch.float).to(device)
        prediction[:, 33] = 1.0
        predictions.append(prediction.unsqueeze(1))

        context =  torch.zeros(values[0,:,:].shape).to(device)
        # print("Decoder: ", max_len)
        for i in range(max_len - 1):
            # * Implement Gumble noise and teacher forcing techniques
            # * When attention is True, replace values[i,:,:] with the context you get from attention.
            # * If you haven't implemented attention yet, then you may want to check the index and break
            #   out of the loop so you do you do not get index out of range errors.

            if (isTrain):
                if (random.uniform(0,1) > teach_prob):
                  char_embed = embeddings[:,i,:]
                else:
                  prediction = torch.distributions.gumbel.Gumbel(prediction.to('cpu'), torch.Tensor([0.1])).sample().to(device)
                  char_embed = embedded_dropout(self.embedding, prediction.argmax(dim=-1))
            else:
                char_embed = self.embedding(prediction.argmax(dim=-1))

            inp = torch.cat([char_embed, context], dim=1)
            hidden_states[0] = self.lstm1(inp, hidden_states[0])

            inp_2 = hidden_states[0][0]
            hidden_states[1] = self.lstm2(inp_2, hidden_states[1])

            ### Compute attention from the output of the second LSTM Cell ###
            output = hidden_states[1][0]
            context, a = self.attention(output, key, values, lens)

            prediction = self.character_prob(torch.cat([output, context], dim=1))
            predictions.append(prediction.unsqueeze(1))

        return torch.cat(predictions, dim=1)

In [ ]:
class Seq2Seq(nn.Module):
    '''
    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.
    This is simply a wrapper "model" for your encoder and decoder.
    '''
    def __init__(self, input_dim, vocab_size, hidden_dim, value_size=128, key_size=128, isAttended=False):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, value_size=value_size, key_size=key_size)
        self.decoder = Decoder(vocab_size, hidden_dim, value_size=value_size, key_size=key_size, isAttended=isAttended)

    def forward(self, speech_input, speech_len, text_input=None, isTrain=True, teach_prob=0.3):
        if isTrain == True:
            key, value, decoder_lens = self.encoder(speech_input, speech_len)
        else:
            key, value, decoder_lens = self.encoder(speech_input, speech_len, isTrain=False)
        if (isTrain == True):
            predictions = self.decoder(key, value, decoder_lens, text_input, teach_prob=teach_prob)
        else:
            predictions = self.decoder(key, value, decoder_lens, text=None, isTrain=False)
        return predictions


In [ ]:
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)
    if type(m) == nn.LSTM:
            for name, param in m.named_parameters():
                    if 'bias' in name:
                        nn.init.constant(param, 0.0)
                    elif 'weight' in name:
                        nn.init.xavier_normal(param)

In [ ]:
# initialize things
model = Seq2Seq(input_dim=40,vocab_size=len(LETTER_LIST),hidden_dim=256,isAttended=True)
model.apply(init_weights)
criterion = nn.CrossEntropyLoss(reduction="sum", ignore_index=0).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.85)
model = model.to(device)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  import sys


In [ ]:
# function to train 1 epoch
def train_epoch(model, train_loader, criterion, optimizer, teach_prob=0.7):
    model.train()
    running_loss = 0.0
    torch.cuda.empty_cache()

    start_time = time.time()
    for batch_idx, (data, target, data_lens, target_lens) in enumerate(train_loader):
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        data_lens = data_lens.to(device)
        target_lens = target_lens.to(device)


        out = model(data, data_lens, target, teach_prob=teach_prob)

        loss = criterion(out.contiguous().view(-1, len(LETTER_LIST)), target.contiguous().view(-1))
        loss.backward()
        # clip_grad_norm_(model.parameters(), 2)
        lol = loss / torch.sum(target_lens)
        perplexity = torch.exp(lol)
        perplexity = perplexity.item()

        ## Printing
        curr_loss = lol.item()
        running_loss += curr_loss
        if batch_idx % 30 == 0:
            print(str(batch_idx) + " running \tloss: " + str(curr_loss) + "\tperp: " + str(perplexity))
        ## Printing ends
        torch.cuda.empty_cache()
        optimizer.step()
        del loss
        del data
        del target

    end_time = time.time()

    running_loss /= len(train_loader)
    print('Training Loss:\t', running_loss, '\tTime:\t',end_time - start_time, 's', "Last Perp:\t", perplexity )
    return running_loss

In [ ]:
# function to test 1 epoch
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        running_perp = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        distances = []
        for batch_idx, (data, target, data_lens, target_lens) in enumerate(test_loader):
          data = data.to(device)
          target = target.to(device) # all data & model on same device
          data_lens = data_lens.to(device)
          target_lens = target_lens.to(device)

          out = model(data, data_lens, isTrain=False)
          # code to get rid of everything greater than lens
          out = out[:, :target.size(1), :]

          loss = criterion(out.contiguous().view(-1, len(LETTER_LIST)), target.contiguous().view(-1))
          lol =  loss / torch.sum(target_lens)
          perplexity = torch.exp(lol)
          running_perp += perplexity.item()
          running_loss += lol.item()

          # decode
          decoded_prediction = out_to_strings(out)
          decoded_target = []

          decoded_target = [''.join(index2letter[x] for x in seq).replace("<sos>", "").replace("<eos>", "").replace("<pad>", "").strip() for seq in target.tolist()]

          if (batch_idx > (0.8 * len(test_loader))):
            print("decoded_target: ", decoded_target[0])
            print("decoded_prediction: ", decoded_prediction[0])

          batch_distances = [distance(o, l) for o, l in zip(decoded_prediction, decoded_target)]
          distances.extend(batch_distances)


        running_loss /= len(test_loader)
        running_perp /= len(test_loader)
        dist = np.mean(distances)
        print('Testing Loss:\t', running_loss)
        print('Distance:\t', dist)
        return running_loss, dist

In [ ]:
n_epochs = 40
Train_loss = []
Test_loss = []
Test_acc = []
teach_prob = 0.1

for i in range(27, n_epochs+1):
    print("running epoch: " + str(i))
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    test_loss, test_acc = test_model(model, valid_loader, criterion)
    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)
    # saving every other model
    # if i % 2 == 0:
    torch.save(model.state_dict(),'/content/drive/My Drive/HW4P2/Awais/' + "model_baseline" + str(i))
        # torch.save(optimizer.state_dict(),'/content/gdrive/My Drive/deep_learning/hw2p2/' + "optimizer" + str(i) + '_ResNet50_Adam.pt')
    print("saved")
    print('='*20)

running epoch: 27
0 running 	loss: 0.8704079985618591	perp: 2.3878848552703857
30 running 	loss: 0.7431126236915588	perp: 2.1024694442749023
60 running 	loss: 0.440836101770401	perp: 1.5540060997009277
90 running 	loss: 0.5560519099235535	perp: 1.7437744140625
120 running 	loss: 0.5059098601341248	perp: 1.6584938764572144
150 running 	loss: 0.5420714020729065	perp: 1.7195650339126587
180 running 	loss: 0.6355567574501038	perp: 1.8880730867385864
Training Loss:	 0.5665325729810086 	Time:	 410.7350044250488 s Last Perp:	 1.8900437355041504
decoded_target:  MILITARY POLICY WAS TO KEEP THE TRAVEL ROUTES OPEN AND PROTECT THE SETTLED AREAS
decoded_prediction:  MILITARY POLICY WAS TO KEEP THE TRAVEL ROUTES OPEN AND PROTECT THE SETTLED AREAS
Testing Loss:	 2.8815512127346463
Distance:	 7.023508137432188
saved
running epoch: 28
0 running 	loss: 0.45980578660964966	perp: 1.5837664604187012
30 running 	loss: 0.4345836639404297	perp: 1.5443198680877686
60 running 	loss: 0.6668385863304138	perp: 1.

KeyboardInterrupt: ignored

In [ ]:
# 80torch.save(model.state_dict(),'/content/drive/My Drive/deep_learning/hw4p2/' + "model_baseline_9.9" + str(i))
# torch.load(model, '/content/drive/My Drive/HW4P2/Models/model_baseline18')
model.load_state_dict(torch.load('/content/drive/My Drive/HW4P2/Awais/model_baseline27'))

<All keys matched successfully>

In [ ]:
def my_collate_fn_test(batch):
  data = batch
  in_lens = torch.LongTensor([len(i) for i in data])
  return pad_sequence(data), in_lens

In [ ]:
test_dataset = Speech2TextDataset(letter2index, speech_test, None, False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, collate_fn=my_collate_fn_test)

In [ ]:
def predict_model(model, test_loader):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        running_perp = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        predictions = []
        for batch_idx, (data, data_lens) in enumerate(test_loader):
          data = data.to(device)
          data_lens = data_lens.to(device)
          # print(data.shape, data_lens.shape)
          # break
          out = model(data, data_lens, isTrain=False)
          # code to get rid of everything greater than lens
          # out = out[:, :target.size(1), :]

          # decode
          decoded_prediction = out_to_strings(out)

          predictions.extend(decoded_prediction)

        return predictions

In [ ]:
labels = predict_model(model, test_loader)
#

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru

In [ ]:
labels

["THE COMPANY ALSO DONATED FIFTY THOUSAND DOLLARS TO A LOCAL ANVIREED METAL GROUP TO HIGHER SPECIALIST TO AVALUERATE THE COMPANY'S PLANS TO CLEAN UP CONTAMINATED SITES",
 "MR. BENET'S CALL FOR MANDATORY TASTING HASN'T BEEN ENDORSED BY THE WHITE HOUSE PERIOD",
 'UNUSERRELE SISSUED YESTERDAY SAID THAT THE LINE OF CREDIT WOULD BE GUARENTEED BY PROVIDEN SOVER PERIOD',
 'THE BUY O PLAN IS CONTENGENT ON ALLEGANINY RECEIVING THAT WAVER PERIOD',
 'HOWEVER INCREASING THE COST OF RESEARCH ANIMALS SHOULD MOT OF AIG RESEARCHERS NOT TO WASE THEM ON MERRLY CARIOUS OR REPEDDITIVE STUDIES',
 "IN AUGUST COMMA A PACIFIC TELECIS GROUP UNIT MADE A SIMILAR GUARANTE TO THE COMMUNICATION' WORKERS OF AMERICA COMMA PROVIDING TRAINING FOR NE POSITIONS OF NECESSARY PERIOD",
 'AN AMBISIOUS NINETEEN HYPHEN NATION EFFORT TO HELPED TWENTY EERS OF WARRNN TEMBODIA ENDED WITHOUT IN AGREEMENT PERIOD',
 'THE NEW CHAIRMAN KNOWS THAT IT POLITICAL CONNECTIONS MAKE SOME MARKET PARTICIPANTS SUSPICIOUS THAT HE WILL TRIED A STI

In [ ]:
saved = np.array([np.arange(len(labels)),labels]).T
np.savetxt('/content/sub13.csv', saved, delimiter=',',fmt='%s')

In [ ]:
!rm -r '/content/drive/My Drive/HW4P2/Models'